# Adversarial example with multinomial logistic regression

In this example, we redo the [adversarial example](https://gurobi-optimization-ml2gurobi.readthedocs-hosted.com/en/latest/examples/adversarial_mnist.html) of the documentation but use a logistic regression model for the regression

We don't detail the optimization model here. Please refer to the example in the documentation.

## Import the necessary packages and load data

We import all the package we need for this example.
We fetch the MINST data set using sklearn's functionalities.

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import gurobipy as gp

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../src')
from gurobi_ml import add_predictor_constr

In [ ]:
# Get MNIST digit recognition data set
# Load data from https://www.openml.org/d/554
X, y = fetch_openml(
    "mnist_784", version=1, return_X_y=True, as_frame=False, parser="pandas"
)

Flatten and scale the data

In [ ]:
X = X.astype(np.float64)
clf = LogisticRegression(C=50.0 / 500, penalty="l1", solver="saga", tol=0.1)
pipeline = make_pipeline(StandardScaler(), clf)
pipeline.fit(X, y)

In [ ]:
imageno = 20

In [ ]:
ex_prob = clf.predict_proba(X[imageno:imageno+1, :])
sorted_labels = np.argsort(ex_prob)[0]
right_label = sorted_labels[-1]
wrong_label = sorted_labels[-2]

In [ ]:
ex_prob.shape

In [ ]:
from gurobi_ml.sklearn.logistic_regression import LogisticRegressionConstr

In [ ]:
pwl_attr = LogisticRegressionConstr.default_pwl_attributes()
pwl_attr

In [ ]:
pwl_attr['FuncPieces']=-1
pwl_attr['FuncPieceError']=1e-2
pwl_attr

In [ ]:
image = X[imageno, :]
plt.imshow(image.reshape((28, 28)), cmap="gray")
label = clf.predict(image.reshape(1,-1))
print(f"Solution is classified as {label}")

In [ ]:


m = gp.Model()
delta = 5*255

x = m.addMVar(image.shape, lb=0.0, ub=255.0, name="x")
y = m.addMVar(ex_prob.shape, lb=-0, ub=1, name="y")

abs_diff = m.addMVar(image.shape, lb=0, ub=255.0, name="abs_diff")

m.addConstr(y[0, wrong_label] >= y[0, right_label] + 1e-3)


# Bound on the distance to example in norm-1
m.addConstr(abs_diff >= x - image)
m.addConstr(abs_diff >= -x + image)
m.addConstr(abs_diff.sum() <= delta)

pred_constr = add_predictor_constr(m, pipeline, x, y, pwl_attributes=pwl_attr, epsilon=0, output_type='probability')

pred_constr.print_stats()

In [ ]:
def simple_obbt(logreg_constr: LogisticRegressionConstr):
    """ Do OBBT on intermediate terms on which exponential is taken
    
    This is a naive implementation of OBBT for those terms.
    """
    m = logreg_constr.gp_model
    m.update()
    with m.relax() as obbt_prob:
        obbt_prob.Params.OutputFlag = 0
        obbt_vars = obbt_prob.getVars()
        # Remove quadratic constraints
        obbt_prob.remove(obbt_prob.getQConstrs())

        coefs = logreg_constr.predictor.coef_
        intercept = logreg_constr.predictor.intercept_
        affinevars = logreg_constr.affinevars

        obbt_prob.optimize()

        for index in np.ndindex(affinevars.shape):
            var = affinevars[index].item()
            obbbt_var = obbt_vars[var.index]

            for sense in (gp.GRB.MINIMIZE, gp.GRB.MAXIMIZE):
                obbt_prob.setObjective(obbbt_var, sense)
                obbt_prob.optimize()

                objval = obbt_prob.objval

                if sense == gp.GRB.MINIMIZE:
                    lower_bound = var.LB
                    new_lower_bound = max(lower_bound, obbt_prob.ObjVal)
                else:
                    upper_bound = var.UB
                    new_upper_bound = min(upper_bound, obbt_prob.ObjVal)
            var.UB = new_upper_bound
            var.LB = new_lower_bound
            print(f"Variable {var.VarName}, current bounds [{lower_bound:5.3}, {upper_bound:5.3}] strengthened [{new_lower_bound:5.3}, {new_upper_bound:5.3}]")
                
simple_obbt(pred_constr[-1])


In [ ]:
m.Params.NonConvex = 2
m.Params.Obbt = 3
m.Params.PreSos1BigM = 1e6
m.Params.PreSos2BigM = 1e6
m.Params.PreSos2Encoding = 3
m.optimize()

In [ ]:
pred_constr.output_values

In [ ]:
np.max(np.abs(pred_constr.output_values - pipeline.predict_proba(pred_constr.input_values)))

In [ ]:
plt.imshow(x.X.reshape((28, 28)), cmap="gray")
label = pipeline.predict(x.X.reshape(1,-1))
print(f"Solution is classified as {label}")

copyright © 2023 Gurobi Optimization, LLC